In [5]:
from networkit import *
import glob
from tqdm import tqdm_notebook
import networkit as nx
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import datetime
import pickle5 as pickle
from glob import glob
from datetime import datetime
import numpy as np

In [2]:
nx.setNumberOfThreads(20)
nx.setLogLevel("TRACE")

In [3]:
with open('../user_details.json') as fp:
    dict_users_original = json.load(fp)
with open('../Networks/interaction_network_per_month.json', 'rb') as fp:
    data = pickle.load(fp)
with open('../Networks/kcore_temportal.json', 'rb') as fp:
    monthwise_kcore_dict = pickle.load(fp)  
with open('../Networks/map_dict_kcore.json', 'rb') as fp:
    map_dict_kcore = pickle.load(fp)  
    
df_user_label=pd.read_csv('../Results/user_labelling.csv')
all_normal_dict={key:1 for key in list(df_user_label[df_user_label['label']=='normal']['user_key'])}
# df_matched=pd.read_csv('../Results/matched.csv')
df_matched=pd.read_csv('../Results/matched_fear_hate.csv')


### Folllower following ratio

### Average messages per day

In [6]:
dict_users={}
files = sorted(glob('../../Gab_Data/new_features_old_gab/gab_fear_hate_features*.pickle'))
dict_date={}

for file in tqdm_notebook(files,total=len(files)):
    
    with open(file, 'rb') as handle:
        Gab_keyword_match = pickle.load(handle)
    
    if('predicted_probab' not in Gab_keyword_match[1].keys()):
        continue
    for element in Gab_keyword_match:
        
        
        try:
            dt = datetime.fromisoformat(element['post_create_time'])
            key_new=str(dt.day)+'/'+str(dt.month)+'/'+str(dt.year)
            
            try:
                dict_date[key_new]+=1
            except KeyError:
                dict_date[key_new]=1
            
        except KeyError:
            continue

        
        try:
            dict_users[element['username']][key_new]+=1
        except KeyError:
            try:
                dict_users[element['username']][key_new]=1
            except KeyError:
                dict_users[element['username']]={}
                dict_users[element['username']][key_new]=1
        
#         try:
#             for label in labels:
#                 dict_users_ambi[element['username']][label]+=1
#             dict_users_ambi[element['username']]['total']+=1
#         except KeyError:
#             dict_users_ambi[element['username']]={'fearspeech':0,'hatespeech':0,'normal':0,'total':0}
#             for label in labels:
#                 dict_users_ambi[element['username']][label]+=1
#             dict_users_ambi[element['username']]['total']+=1

            

/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [7]:
for user in tqdm_notebook(dict_users, total=len(dict_users)):
    for date in dict_date:
        try:
            temp=dict_users[user][date]
        except KeyError:
            dict_users[user][date]=0
        

/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [9]:
average_messages=[]
for user in dict_users:
    avg_message=np.mean(list(dict_users[user].values()))
    average_messages.append(avg_message)
    

In [10]:
count=0
for avg_msg in average_messages:
    if(avg_msg>3):
        count+=1

In [11]:
count/len(average_messages)

0.012896360681714349

In [ ]:
dict_users_filter={}
max_fear=0
max_hate=0
for key in dict_users.keys():
    user=dict_users[key]
    total=user['fearspeech']+user['hatespeech']
    
    if(user['fearspeech']>max_fear):
        max_fear=user['fearspeech']
    
    if(user['hatespeech']>max_hate):
        max_hate=user['hatespeech']
    
    if(total>=10):
        dict_users_filter[key]=user
        dict_users_filter[key]['name']=key
